### Function Calling and API Calls

A lot of projects with Langchain are just scripts executed on a local computer. In the "real world", production ready
services run in some kind of standardized wrapper, like an API inside a Docker container. This is an approach how you might
interact with a REST-API with an LLM Chain (which might later also be used inside an API withh a single POST endpoint)

In [8]:
from dotenv import load_dotenv
import os

load_dotenv()
API_KEY = os.environ.get("API_KEY")

In [9]:
import requests
import openai

def get_todos(completed=None):
    params = {'completed': completed} if completed is not None else None
    response = requests.get('https://fastapilangchain-1-w0858112.deta.app/todos', params=params)
    return response.json()

def create_todo(todo):
    response = requests.post('https://fastapilangchain-1-w0858112.deta.app/todos', json=todo)
    return response.json()


In [10]:
functions = [
    {
        "name": "get_todos",
        "description": "Get a list of todos, optionally filtered by their completion status",
        "parameters": {
            "type": "object",
            "properties": {
                "completed": {
                    "type": "boolean",
                    "description": "Whether to only return completed todos",
                },
            },
            "required": [],
        },
    },
    {
        "name": "create_todo",
        "description": "Create a new todo",
        "parameters": {
            "type": "object",
            "properties": {
                "todo": {
                    "type": "object",
                    "description": "The new todo to be created",
                    "properties": {
                        "id": {
                            "type": "integer",
                            "description": "The id of the todo",
                        },
                        "task": {
                            "type": "string",
                            "description": "The task of the todo",
                        },
                        "is_completed": {
                            "type": "boolean",
                            "description": "Whether the task is completed",
                            "default": False
                        },
                    },
                    "required": ["task"],
                },
            },
            "required": ["todo"],
        },
    }
]

In [13]:
query="I want to walk my dog in the afternoon"

response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": query}],
        functions=functions,
    )
message = response["choices"][0]["message"]

message

<OpenAIObject at 0x21fbcd80470> JSON: {
  "content": null,
  "function_call": {
    "arguments": "{\n  \"todo\": {\n    \"id\": 1,\n    \"task\": \"Walk the dog in the afternoon\",\n    \"is_completed\": false\n  }\n}",
    "name": "create_todo"
  },
  "role": "assistant"
}

In [15]:
if message.get("function_call"):
    function_name = message["function_call"]["name"]


function_name

'create_todo'